# Chapter3-4

## ライブラリのインストール

In [1]:
!pip install transformers[ja,sentencepiece,torch] pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 697.8 kB/s e

## サブワード語彙を構築する
今回は疑似テキストとして以下のような文を使用する<br>
```txt
たのしいたのしいたのしいたのしいたのしいたのしいたのしさたのしさ
うつくしいうつくしいうつくしいうつくしいうつくしさ
```

※単語の境界を越えたサブワードの組の結合は今回は行わないものとする

In [5]:
# 単語とその頻度を変数を使用して初期化
word_freqs = {
    "たのしい": 6,
    "たのしさ": 2,
    "うつくしい": 4,
    "うつくしさ": 1,
}

print(f"word_freqs: {word_freqs}")

# 語彙を文字で初期化
vocab = sorted(set([char for word in word_freqs for char in word]))

# 単語とその分割の状態
splits = {word: [char for char in word] for word in word_freqs}

print(f"vocab: {vocab}")
print(f"splits: {splits}")

word_freqs: {'たのしい': 6, 'たのしさ': 2, 'うつくしい': 4, 'うつくしさ': 1}
vocab: ['い', 'う', 'く', 'さ', 'し', 'た', 'つ', 'の']
splits: {'たのしい': ['た', 'の', 'し', 'い'], 'たのしさ': ['た', 'の', 'し', 'さ'], 'うつくしい': ['う', 'つ', 'く', 'し', 'い'], 'うつくしさ': ['う', 'つ', 'く', 'し', 'さ']}


## バイト対符号化を行うための関数を定義する

`compute_most_frequent_pair`: 頻度の高い隣接するサブワードの組を計算<br>
`merge_pair`: サブワードの組を結合する

In [7]:
from collections import Counter

def compute_most_frequent_pair(
    splits: dict[str, list[str]]
) -> tuple[str, str]:
  """
  最も頻度の高い隣接するサブワードの組を計算する
  """
  # サブワードの組のカウンタ
  pair_freqs = Counter()

  # すべての単語を処理
  for word, freq in word_freqs.items():

    # 現在の単語の分割状態を取得する
    split = splits[word]

    print(f"split: {split}")

    # すべての隣接したサブワードの組を処理する
    for i in range(len(split) - 1):
      pair = (split[i], split[i + 1])

      pair_freqs[pair] += freq

  print(f"pair_freqs: {pair_freqs}")

  # カウンタから最も頻度の高いサブワードの取得
  pair, _ = pair_freqs.most_common(1)[0]
  return pair

def merge_pair(
    target_pair: tuple[str, str], splits: dict[str, list[str]]
) -> dict[str, list[str]]:

  """
  サブワードの組を結合する
  """
  print(f"target_pair: {target_pair}")

  l_str, r_str = target_pair

  for word in word_freqs:
    split = splits[word]

    print(f"split: {split}")

    i =  0

    # すべての隣接したサブワードの組を処理する
    while i < len(split) - 1:

      # サブワードの組が結合対象と一致したら結合
      if split[i] == l_str and split[i + 1] == r_str:
        split = split[:i] + [l_str + r_str] + split[i + 2 :]
      i += 1

    splits[word] = split

  return splits


## 結合回数を指定しバイト対符号化の語彙を計算する

今回の結合回数は9回とする

In [9]:
for step in range(9):
  # 最も頻度の高い隣接するサブワードの組を計算
  target_pair = compute_most_frequent_pair(splits)

  # サブワードの組を結合
  splits = merge_pair(target_pair, splits)

  # サブワードの組を追加する
  vocab.append(target_pair)

split: ['た', 'の', 'し', 'い']
split: ['た', 'の', 'し', 'さ']
split: ['う', 'つ', 'く', 'し', 'い']
split: ['う', 'つ', 'く', 'し', 'さ']
pair_freqs: Counter({('し', 'い'): 10, ('た', 'の'): 8, ('の', 'し'): 8, ('う', 'つ'): 5, ('つ', 'く'): 5, ('く', 'し'): 5, ('し', 'さ'): 3})
target_pair: ('し', 'い')
split: ['た', 'の', 'し', 'い']
split: ['た', 'の', 'し', 'さ']
split: ['う', 'つ', 'く', 'し', 'い']
split: ['う', 'つ', 'く', 'し', 'さ']
split: ['た', 'の', 'しい']
split: ['た', 'の', 'し', 'さ']
split: ['う', 'つ', 'く', 'しい']
split: ['う', 'つ', 'く', 'し', 'さ']
pair_freqs: Counter({('た', 'の'): 8, ('の', 'しい'): 6, ('う', 'つ'): 5, ('つ', 'く'): 5, ('く', 'しい'): 4, ('し', 'さ'): 3, ('の', 'し'): 2, ('く', 'し'): 1})
target_pair: ('た', 'の')
split: ['た', 'の', 'しい']
split: ['た', 'の', 'し', 'さ']
split: ['う', 'つ', 'く', 'しい']
split: ['う', 'つ', 'く', 'し', 'さ']
split: ['たの', 'しい']
split: ['たの', 'し', 'さ']
split: ['う', 'つ', 'く', 'しい']
split: ['う', 'つ', 'く', 'し', 'さ']
pair_freqs: Counter({('たの', 'しい'): 6, ('う', 'つ'): 5, ('つ', 'く'): 5, ('く', 'しい'): 4, ('し', 'さ'): 3, ('た

In [10]:
# サブワードの組を表示する
print(vocab)

['い', 'う', 'く', 'さ', 'し', 'た', 'つ', 'の', ('し', 'い'), ('た', 'の'), ('たの', 'しい'), ('う', 'つ'), ('うつ', 'く'), ('うつく', 'しい'), ('し', 'さ'), ('たの', 'しさ'), ('うつく', 'しさ')]


## 単語が空白で区切られない言語の扱い

日本語、韓国語、中国語などの単語が空白で区切られない言語の扱いには工夫が必要<br>

多言語BERTでは漢字などは文字単位で分割をされてしまう

In [14]:
from transformers import AutoTokenizer

mbert_tokenizer = AutoTokenizer.from_pretrained(
    "bert-base-multilingual-cased"
)

# 漢字をサブワード分割した結果を出力してみる
print(mbert_tokenizer.tokenize("自然言語処理"))

['自', '然', '言', '語', '処', '理']


In [15]:
# 別の文を使用して実装してみる
print(mbert_tokenizer.tokenize("自然言語処理にディープラーニングを使用する"))

['自', '然', '言', '語', '処', '理', 'に', '##ディ', '##ープ', '##ラー', '##ニング', '##を', '使', '用', 'する']


"##"はBERT・多言語BERTにおいて単語の途中から始まるサブワードを表している

## XLM-R, mT5を使用した場合

文ベースのバイト対符号化によってより自然な分割を行うことができる

In [16]:
xlmr_tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
print(xlmr_tokenizer.tokenize("自然言語処理にディープラーニングを使う"))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

['▁', '自然', '言語', '処理', 'に', 'ディー', 'プラ', 'ー', 'ニング', 'を使う']


文ベースでのバイト対符号化は分割の結果が自然な単語境界に従わないことがあり、<br>単語単位の自然言語処理のタスクを解くことは困難


In [17]:
print(xlmr_tokenizer.tokenize("僕は日本で生まれました"))

['▁', '僕は', '日本で', '生まれ', 'ました']


In [18]:
print(xlmr_tokenizer.tokenize("本日はよろしくお願いいたします"))

['▁本', '日は', 'よろしくお願いいたします']


### 解決策
形態素解析器を用いて単語単語に分割する
MeCabなどがそれにあたる

In [19]:
# 形態素解析器が導入されているモデルを使ってみる
bert_ja_tokenizer = AutoTokenizer.from_pretrained(
    "cl-tohoku/bert-base-japanese-v3"
)

print(bert_ja_tokenizer.tokenize("自然言語処理にディープラーニングを使用する"))

tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

['自然', '言語', '処理', 'に', 'ディープ', 'ラー', '##ニング', 'を', '使用', 'する']


In [21]:
# 別の言葉でも実行してみる
print(bert_ja_tokenizer.tokenize("私は日本出身です"))

['私', 'は', '日本', '出身', 'です']
